# Optimal Methods from Dimitris Bertsimas

We start with OCT, then OCT-H, and then we move to training a neural network and apporximate it with an OCT-H.
We also perform hardest subset selection to train our model and use a multimodal dataset we have created from text embeddings contained in the provided data. 

In [1]:
# Install packages
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("CategoricalArrays")
Pkg.add("Plots")
Pkg.add("Statistics")
Pkg.add("Graphviz_jll") 
import Pkg; Pkg.add(url="https://github.com/InterpretableAI/IAISystemImages.jl") # from the installation page:  https://docs.interpretable.ai/stable/installation/

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
    Updating git-repo `https

In [2]:
using CSV, DataFrames, CategoricalArrays, Plots, Statistics, IJulia, Graphviz_jll  

### Dataset Loading

In [ ]:
data = CSV.read("train.csv", DataFrame, pool=true);


In [ ]:
# Train OCTs with depth=3 and minbucket=5, using each of the loss functions (misclassification, gini, and entropy). 
# Discuss the differences in the resulting trees and report the AUC performance on the validation set.
# https://docs.interpretable.ai/stable/IAIBase/learner/#Common-learner-parameters

seed = 15095

# Choose your positive class label for AUC (change if your labels differ)
# const POS_LABEL = 1.0

val_auc_1 = Float64[]

# TODO: write array of loss functions that we will experiment with
loss_functions = [:misclassification, :gini, :entropy]

best_models = Dict{Symbol, IAI.Learner}()  

# Experiment with different loss functions
for loss_fn in loss_functions
    
    # Base learner fixed to depth=3 and minbucket=5 (and cp=0 to disable pruning)
    base = IAI.OptimalTreeClassifier(
        random_seed = seed,
        max_depth   = 3,
        minbucket   = 5,
        cp          = 0.0,
    )
    # Set the split criterion for this run
    base.criterion = loss_fn

    # (Optional) set any other params here if needed
    # set_params!(base; some_param = some_value)

    # GridSearch: use 5-fold CV to evaluate (no tuning ranges since depth & minbucket are fixed)
    grid = IAI.GridSearch(
        base;
        n_folds = 5,
        validation_criterion = :auc,
        fit_kwargs = (positive_label = POS_LABEL,)
    )

    # Get learner via GridSearch
    IAI.fit!(grid, X_train, y_train)

    # Get the best learner
    best_model = IAI.get_learner(grid)
    # Optionally also access best parameters again if needed
    best_param = IAI.get_best_params(grid)

    # Save the best model for this loss in a dictionary
    best_models[loss_fn] = best_model

    # Get AUC score for validation data (name the variable holding the AUC score as `val_auc`)
    val_auc = IAI.score(best_model, X_valid, y_valid; criterion = :auc, positive_label = POS_LABEL)
    println("[$(loss_fn)] Validation AUC: ", round(val_auc, digits=4))

    # Add AUC score to AUC tracker
    push!(val_auc_1, val_auc)
end